# Train Model
## SageMaker Script Mode


In [2]:
import os
import tensorflow as tf

In [3]:
# pre-TensorFlow 2.0
# set up eager execution
tf.enable_eager_execution()
tf.set_random_seed(0)
tf.logging.set_verbosity(tf.logging.ERROR)

## Global Constants

In [45]:
S3_TFRECORDS_PATH = "s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/tfrecords/"
TFRECORDS_TARBALL = "20190718_tfrecords.tar.gz"

## Data



In [46]:
s3_tfrecords = os.path.join(S3_TFRECORDS_PATH, TFRECORDS_TARBALL)
print (s3_tfrecords)
! aws s3 cp $s3_tfrecords data   

s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/tfrecords/20190718_tfrecords.tar.gz
download: s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/tfrecords/20190718_tfrecords.tar.gz to data/20190718_tfrecords.tar.gz


In [48]:
! tar -xvf data/$TFRECORDS_TARBALL --strip=1 -C data/tfrecords

20190718_tfrecords/test.tfrecord
20190718_tfrecords/train.tfrecord
20190718_tfrecords/val.tfrecord
